# Text content generation using LSTMs

LSTM (Long Short Term Memory networks) are one of the building blocks of RNNs (Recurrent Neural Networks). Unlike traditional neural networks that have fixed input sizes, LSTMs can be fed a varying lenght sequence of inputs. This makes them perfect for dealing with many text related applications.

One application where LSTMs are ["unreasonably effective"](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) is text generation. Basically, we start with a corpus of text (like a set of blog posts, letters, tweets, or even source code) and train a neural network over it. Once trained, we can feed some starter text to the network, and have it predict the next character. Repeat this process N times with a sliding window, and you can generate N characters of text.

This notebook is an example of applying a very simple LSTM (built with Keras & Tensorflow) over a corpus of transcribed podcast content.

### Topics covered in this notebook
1. Pre-processing text data input for LSTM.  
1. Using trained model to generate text while varying temperature parameter.  
1. Keras sequence generators.  
1. Keras callbacks.  

## Pre-requisites

### Software
This tutorial assumes that you have a workstation with Tensorflow, Keras and some other common data science utilities installed. Since LSTMs are quite compute-intensive to train, running this example without a GPU is mostly impractical.

I like to install these tools in a separate python [virtualenv](http://docs.python-guide.org/en/latest/dev/virtualenvs/). This keeps things neat and clean.

I configure my virtual environment as follows:

```shell
mlpy3venv() {
  if [ -d "~/Venv" ] ; then
    mkdir ~/Venv
  fi
  virtualenv --system-site-packages -p python3 ~/Venv/mlpy3venv
  source ~/Venv/mlpy3venv/bin/activate
  easy_install -U pip
  pip3 --no-cache-dir install \
    Pillow \
    h5py \
    ipykernel \
    jupyter \
    matplotlib \
    numpy \
    pandas \
    scipy \
    sklearn \
    keras \
    tqdm \
    && \
    python -m ipykernel.kernelspec
  pip3 install --upgrade tensorflow-gpu
  deactivate

  cat >> ~/.bashrc <<EOF
mlpy3 () {
  cd ~/Project
  source ~/Venv/mlpy3venv/bin/activate
}
EOF
}

mlpy3venv
```

This creates a python 3 virtual env in `~/Venv/mlpy3venv`, installs necessary libraries in it, and creates a bash alias **mlpy3** to quickly activate and use it.

For more details, you can refer my complete workstation setup script [here](https://github.com/sumitgouthaman/workstation-setup/blob/master/workstation_setup.sh).

Run the cell below to import necessary libraries to run rest of the notebook.

**Note about python 2/3:** I try to be both python 2 and 3 compatible for rest of the code in this notebook.

In [1]:
# Python 2 / 3 compatibility
from __future__ import print_function
from __future__ import division

from datetime import datetime
import io
import math
import os
import pickle
import random
import re
import string
import sys

from tqdm import tqdm  # To draw progress bars

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  

# Keras
from keras.callbacks import ModelCheckpoint, LambdaCallback, ReduceLROnPlateau
from keras.layers import Dense, Activation, Dropout, LSTM, Bidirectional                                                
from keras.models import Sequential                 
from keras.optimizers import RMSprop, Adam                
from keras.preprocessing.text import text_to_word_sequence
from keras.utils import Sequence
from keras.utils.data_utils import get_file 
from keras import backend as K

# Scikit learn
from sklearn.model_selection import train_test_split

/home/sumit/Venv/mlpy3venv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Choosing a corpus

I chose to use TWiT's [Security Now](https://twit.tv/shows/security-now) podcast content as the corpus for this experiment. The reasons are pretty straight-forward:  

- Security Now has over 600 episodes spanning more than a decade of stories.  
- Steve Gibson hosts meticulously transcribed content of each episode in plain text format on [GRC](https://www.grc.com/securitynow.htm).  

I am not aware of any other Podcast that meets the same criteria. I considered using off-the-shelf speech-to-text APIs to transcribe other podcasts, but the time and money involved in doing so seemed excessive for a small experiment.

We use Keras' built-in `get_file()` function to download and cache episode content as text files locally.

In [2]:
latest_episode_number = 646  # From: https://www.grc.com/securitynow.htm

# Attempt to download as much of Security Now transcript if possible
downloaded_episode_files = []
for episode in range(1, latest_episode_number+1):
    source_path = 'https://www.grc.com/sn/sn-%03d.txt' % episode
    dest_file_path = '%03d.txt' % episode
    try:
        path = get_file(dest_file_path, origin=source_path)
        downloaded_episode_files.append(path)
    except Exception as e:
        # Ignore: Some episodes might not exists
        print('Downloading episode %03d. Episode might not exist.' % episode)

print('Episodes downloaded: %d' % len(downloaded_episode_files))
print('First 5:\n%s' % ',\n'.join(downloaded_episode_files[:5]))

Episodes downloaded: 641
First 5:
/home/sumit/.keras/datasets/001.txt,
/home/sumit/.keras/datasets/002.txt,
/home/sumit/.keras/datasets/003.txt,
/home/sumit/.keras/datasets/004.txt,
/home/sumit/.keras/datasets/005.txt


## Pre-processing

As with any machine learning problem, the first step is usually to peek at the dataset and clean it up for consumption by our algorithm.

### Non ascii characters
A preliminary examination of the files showed me that there are some non-ascii characters in it. We should probably get rid of them to keep things simpler.

We can use the `.encode('ascii', 'ignore')` construct to get rid of all the non-ascii characters. In python 2, this returns a string. However, in python 3, this returns a byte array, and we need to add a `decode('ascii')` to keep the code python 2/3 compliant.

We also convert the text to **lower-case** for simplification.

We print out a small sample from the first and last episode for a sanity check.

In [3]:
episode_texts = []
for path in downloaded_episode_files:
    # Load episode texts.
    # Also convert to  lower case and remove non-ascii characters.
    episode_texts.append(
        io.open(path, encoding='ISO-8859-1')
        .read()
        .lower()
        .encode('ascii', 'ignore')
        .decode('ascii')
    )
    
print('Sample episode text:')
print('-' * 50)
print(episode_texts[0][:1000])  # First episode
print('-' * 50)
print(episode_texts[-1][:1000])  # Last episode
print('-' * 50)

Sample episode text:
--------------------------------------------------
gibson research corporation	http://www.grc.com/

series:		security now!
episode:	#1
date:		august 19, 2005
title:		as the worm turns: the first internet worms of 2005
speakers:	steve gibson & leo laporte
source file:	http://media.grc.com/sn/sn-001.mp3
file archive:	http://www.grc.com/securitynow.htm

description:  how a never-disclosed windows vulnerability was quickly reverse-engineered from the patches to fix it and turned into more than 12 potent and damaging internet worms in three days.  what does this mean for the future of internet security?

leo laporte:  hi, this is leo laporte, and i'd like to introduce a brand-new podcast to the twit lineup, security now! with steve gibson.  this is episode 1 for august 18, 2005.  you all know steve gibson.  he, of course, appears on twit regularly, this week in tech.  we've known him for a long time.  he's been a regular on the screensavers and call for help.  and, you 

### Stripping metadata line

We can see that the file contains some metadata about the episode at the beginning. We are primarily interested in generating the podcast content, so we should strip this off. A manual inspection of the first few lines of random episodes indicates that the size of this metadata is consistent across all episodes. We typically have 12 lines of intro metadata at the start and around 4 lines of footer at the end.

We use strip_intro_lines() to remove these lines from all the episodes.

As before, we print out some processed episodes for a sanity check.

In [4]:
def strip_intro_lines(episode_text, lines_to_strip_head, lines_to_strip_tail):
    """Return episode text after removing the first #lines_to_strip lines.
    
    Args:
        episode_text: The text of the episode.
        lines_to_strip: Number of lines to remove from the start of the episode.
        
    Returns:
        The episode text with lines removed.
    """
    lines = episode_text.split('\n')
    lines = lines[lines_to_strip_head:-lines_to_strip_tail]
    return '\n'.join(lines)

# The beginning 12 lines need to be stripped (show metadata).
# The last 4 lines need to be stripped (copyright).
episode_texts = [strip_intro_lines(e, 12, 4) for e in episode_texts]

print('Sample episode text after processing:')
print('-' * 50)
print(episode_texts[0][:1000])  # First episode
print('-' * 50)
print(episode_texts[-1][:1000])  # Last episode
print('-' * 50)

Sample episode text after processing:
--------------------------------------------------
leo laporte:  hi, this is leo laporte, and i'd like to introduce a brand-new podcast to the twit lineup, security now! with steve gibson.  this is episode 1 for august 18, 2005.  you all know steve gibson.  he, of course, appears on twit regularly, this week in tech.  we've known him for a long time.  he's been a regular on the screensavers and call for help.  and, you know, he's well-known to computer users everywhere for his products.  he's very well known to consumers for spinrite, which was the inspiration for norton disk doctor and still runs rings around it.  it is the ultimate hard-drive diagnostic recovery and file-saving tool.  it's really a remarkable tool that everybody should have a copy of from grc.com.  but he's also been a very active consumer advocate, working really hard to help folks with their security.  he first came to my attention with the click of death, which was - that was 

### Exploring the dataset

We first check what sorts of characters are present in our dataset. We expect it to contain all the letters of the english alphabet with some common punctuations added in.

In [5]:
unique_chars = set()
for episode_text in tqdm(episode_texts):
    chars_in_episode = set(episode_text)
    unique_chars |= chars_in_episode
    
print('Unique chars in dataset: %d' % len(unique_chars))
print(unique_chars)

100%|██████████| 641/641 [00:00<00:00, 1713.04it/s]

Unique chars in dataset: 72
{'l', ';', '*', '3', '2', 'z', '0', 's', '_', 'h', 'u', '=', '#', 'j', '/', 'd', '^', '`', '9', '>', '?', 'a', "'", '-', '(', '8', '7', '.', '{', '[', 'b', 't', '|', '\x1e', '"', 'p', 'y', 'm', '%', '~', 'g', 'x', '$', '1', ' ', 'r', 'c', ',', '\\', '\t', 'k', 'o', 'w', 'n', '!', '@', ']', 'e', '4', '<', ')', '+', 'v', '}', 'f', ':', '\n', '&', '6', 'i', 'q', '5'}


This is mostly what I expected. But there is a strange '\x1e' character thrown in.

Some [preliminary investigation](http://facweb.cs.depaul.edu/sjost/it211/documents/ascii-npr.htm) tells me it's a "Record separator". Strange. I don't know what that is, but I'll leave it there anyways.

## Building the model

I've heard Keras is a great library for beginners. A quick search for "LSTM text generation keras" leads directly to this [file](https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py).

The example uses Nietzsche's writings as the dataset. We can re-use significant portions of the code. But our dataset is an order of magnitude larger, which is going to create some problems as we will see.

### Input format

Lets assume we have a language with 4 unique characters A, B, C and D. Given a text of N characters, we want to predict the (N+1)st character.

We start by assigning each character from our corpus a unique integer id. When feeding input into our model, we will represent this id as a [one-hot](https://en.wikipedia.org/wiki/One-hot) encoded vector.

So assume, in out simplified case where we only have 4 characters, we assign the encodings as follows:

```
A -> 1 -> 0001
B -> 2 -> 0010
C -> 3 -> 0100
D -> 4 -> 1000
```

To proceed, we first assign each of the 72 character in our actual dataset a unique integer id. We store this in a dictionary so that it can be retrieved later. We also store a inverse dictionary mapping the integer id back to the character so that we can decode the output of our model.

In [6]:
unique_chars = sorted(list(unique_chars))
char_indices = dict((c, i) for i, c in enumerate(unique_chars))
indices_char = dict((i, c) for i, c in enumerate(unique_chars))

Going back to our simplified situation with only 4 characters, assume we have a sentence in our corpus as follows:

```
ACDBACCDBDACDABA
```

To feed this into our network, we will chop the sentence into smaller examples that operate over a sliding window.

Assume, we want a window of size 5 with a step size of 2. What this means is that we will feed 5 characters into the network and attempt to predict the next character. For the next example, we will move 2 characters to the right and pick the next 5 characters.

So our original example sentence will produce the following input examples:

```
 [ACDBA](C) CDBDACDABA -> X: ACDBA, Y: C
AC [DBACC](D) BDACDABA -> X: DBACC, Y: D
ACDB [ACCDB](D) ACDABA -> X: ACCDB, Y: D
ACDBAC [CDBDA](C) DABA -> X: CDBDA, Y: C
ACDBACCD [BDACD](A) BA -> X: BDACD, Y: A
ACDBACCDBD [ACDAB](A)  -> X: ACDAB, Y: A
```

Now for each value of X, we encode each character as a one-hot encoded vector. So:

```
ACDBA -> [[0,0,0,1], [0,1,0,0], [1,0,0,0], [0,0,1,0], [0,0,0,1]]
```

So, if our dataset produces N input examples, the total size of X input matrix to our model would be:
```
N x <Window Size> x <Size of charater set>
```

This is a really small example, with a unrealistically small character set. But in reality, this way of representing the input data can lead to very high memory requirements.

Let's try to estimate the size of the input tensor X, if we used our entire dataset with a window size of 100, step size of 2 and character set size of 72.

In [7]:
total_dataset_length = sum(len(e) for e in episode_texts)
assumed_input_window_size = 100
assumed_step_size = 2

estimated_input_examples = len(
    range(
        0,
        total_dataset_length - assumed_input_window_size,
        assumed_step_size
    )
)

print('Total dataset character lenght: %d' % total_dataset_length)
print('Assuming window size of %d and step size of %d:' % (assumed_input_window_size, assumed_step_size))
print('Estimated shape of X: %d x %d x %d' % (estimated_input_examples, assumed_input_window_size, len(unique_chars)))
print('Estimated size of X: %d' % (estimated_input_examples*assumed_input_window_size*len(unique_chars)))

Total dataset character lenght: 45658091
Assuming window size of 100 and step size of 2:
Estimated shape of X: 22828996 x 100 x 72
Estimated size of X: 164368771200


These are some huge numbers and we would definitely not be able to fit the entire dataset in our memory. 

### Solving the memory issue for our input tensor

A solution for this is to avoid holding the entire dataset in an in-memory tensor. Instead we want a way to generate input tensors in batches as and when needed.

We will use a Keras mechanism called [Sequence generators](https://keras.io/utils/#sequence) to achieve this. Implementing a sequence generator involves two methods:

1. A `__len__()` method that returns number of batches.  
2. A `__getitem__(batch_index)` method that returns a particular batch on demand.

We implement a EpisodesSequence class to handle this for us. An object of this class can be initialized by passing in a array of episode texts.

We will reuse this class for both the training sequence and the validation sequence.

In [8]:
class EpisodesSequence(Sequence):

    def __init__(self, episode_texts, window_size, step_size, unique_chars, char_indices, batch_size):
        self.all_episodes_texts = '\n'.join(episode_texts)
        self.window_size = window_size
        self.step_size = step_size
        self.char_indices = char_indices
        self.batch_size = batch_size
        
        self.character_set_length = len(unique_chars)
        self.total_input_examples = int(
            math.ceil((len(self.all_episodes_texts) - self.window_size) / self.step_size)
        )
        self.total_batches = int(
            math.ceil(self.total_input_examples / self.batch_size)
        )
        
        print(
            'EpisodesSequence generator initialized. Total windows: %d, Total batches: %d' %(
                self.total_input_examples, self.total_batches
            )
        )

    def __len__(self):
        return self.total_batches

    def __getitem__(self, batch_index):
        examples_start_index = batch_index * self.batch_size
        examples_end_index = min(examples_start_index+self.batch_size, self.total_input_examples)
        current_batch_size = examples_end_index - examples_start_index
        sentences = []
        next_chars = []
        text_index = examples_start_index * self.step_size  # Start of batch
        for i in range(current_batch_size):
            sentences.append(self.all_episodes_texts[text_index: text_index + self.window_size])
            next_chars.append(self.all_episodes_texts[text_index + self.window_size])
        x = np.zeros((len(sentences), self.window_size, self.character_set_length), dtype=np.bool)
        y = np.zeros((len(sentences), self.character_set_length), dtype=np.bool)
        for i, sentence in enumerate(sentences):
            for t, char in enumerate(sentence):
                x[i, t, char_indices[char]] = 1
            y[i, char_indices[next_chars[i]]] = 1
        return x, y

### Dealing with failures and using checkpointing

Training a LSTM model like the one we intend to build takes a long time. It is prudent to checkpoint your model periodically.

Going one step further, we will implement a helper method that we use during our training step. This method will detect if there are model checkpoint files (.hdf5) lying around in a folder with the same name as the model. If yes, it figures out what was the highest epoch that was checkpointed, restores weights from that checkpoint and continues training from the next epoch.

In [9]:
def initialize_from_last_checkpoint(model):
    model_dirs = [d for d in os.listdir('.') if os.path.isdir(os.path.join('.', d)) and d.startswith(model.name)]
    
    if not model_dirs:
        return 0, None
    
    # Get directory corresponding to last execution (should be one with highest timestamp)
    latest_run_dir = os.path.join('.', sorted(model_dirs)[-1])
    
    # Get list of saved model files
    checkpointed_models = [m for m in os.listdir(latest_run_dir) if m.endswith('.hdf5')]
    
    if not checkpointed_models:
        return 0, None
    
    # Sort by epoch number
    checkpointed_models = sorted(checkpointed_models, key=lambda m: int(re.search('ep-(\d+)', m).group(1)))
    
    # Choose model with highest epoch
    latest_model = checkpointed_models[-1]
    latest_model = os.path.join(latest_run_dir, latest_model)
    epoch = int(re.search('ep-(\d+)', latest_model).group(1))
    
    print('Loading weights from %s' % latest_model)
    model.load_weights(latest_model)
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return epoch, latest_run_dir

### Training

We implement a helper method that runs training for some number of epochs on our dataset. It uses some of the helper methods we defined above to restore previously checkpointed model weights.

We also add a Keras [callback](https://keras.io/callbacks/) that checkpoints the model after each epoch.

In [10]:
def train_model(model, generator, validation_generator, epochs, on_epoch_end,
                continue_from_checkpoint=False, lr_decay=False):
    
    initial_epoch = 0
    checkpoint_folder_name = '%s_%s' % (model.name, datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
    
    if continue_from_checkpoint:
        initial_epoch, prev_folder_name = initialize_from_last_checkpoint(model)
        if prev_folder_name:
            print('Re-using existing folder %s.' % prev_folder_name)
            checkpoint_folder_name = prev_folder_name
    
    print('Checkpoint folder: %s' % checkpoint_folder_name)
    
    if not os.path.isdir(checkpoint_folder_name):
        os.makedirs(checkpoint_folder_name)
        
    lambda_callback = LambdaCallback(on_epoch_end=on_epoch_end)
    filepath='%s/ep-{epoch:02d}-vloss-{val_loss:.3f}.hdf5' % checkpoint_folder_name
    checkpoint_callback = ModelCheckpoint(filepath, monitor='val_loss',verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)
    callbacks_list = [lambda_callback, checkpoint_callback]
    
    if lr_decay:
        decay_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, epsilon=0.001)
        callbacks_list.append(decay_callback)
    
    model.fit_generator(
        generator=generator,
        epochs=epochs,
        callbacks=callbacks_list,
        validation_data=validation_generator,
        shuffle=False,
        initial_epoch=initial_epoch,
    )

### Using the model to generate text

Next we will implement a method to generate text by using our trained model.

The input to our model takes a list of one-hot encoded character vectors. This list is our seed text. The ouput of the model is a list of probabilities for each of the characters in our corpus.

```
[[0,0,0,1], [0,1,0,0], [1,0,0,0], [0,0,1,0], [0,0,0,1]] -> model -> [0.01, 0.03, 0.85, 0.02]
```

The simplest approach would be to choose the character with the highest probability. We then shift the window one step to the right (i.e. our newly predicted character becomes the last character in the next input we feed to the model). But this often leads to text that is repetitive.

The solution is to take into account a temperature parameter that tweaks (scales) our probabilities such that there is an increased likelihood of picking a different character. I learnt this from the keras text generation example code. Later, I found some links that explain what is going on here [[1]](https://stackoverflow.com/questions/37246030/how-to-change-the-temperature-of-a-softmax-output-in-keras), [[2]](https://www.machinelearningpython.org/single-post/Text-Generation---An-Improvement).

We put this toghther in a `generate_sample_text` function that starts with a random seed text of length = window size. We encode this string in our typical input format as a vector of one-hot encoded vectors. Passing this through our model produces an output which can be interpreted as probabilities of each output character. We use our sample method to decide on a predicted character.

At the next step, we shift our window one character to the right and repeat the process all over again. We can continue this till we generate as many characters as we want.

In [11]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_sample_text(model, episode_texts, window_size, unique_chars, char_indices, indices_char):
    
    text = '\n'.join(episode_texts)
    start_index = random.randint(0, len(text) - window_size - 1)

    print('-' * 50)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('-- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + window_size]
        generated += sentence
        print('-- Generating with seed:\n%s\n--' % sentence)

        for i in range(400):
            x_pred = np.zeros((1, window_size, len(unique_chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
        
        print(generated)
        print('-' * 50)

## Trying some models

### Simple LSTM model
This is a very simple model (very similar to the Keras sample code).

1. It has one LSTM layer with 256 hidden units.  
2. We add a dropout of 0.5 to avoid over-fitting.  
3. We add a densely connected layer with output dimension equal to number of characters in our corpus.  
4. We finally end with a softmax layer.

In [12]:
window_size=200
step_size=4
batch_size=256

lstm_layer_size=256

character_set_length = len(unique_chars)

model_name = 'lstm_%d_1layer_wsize_%d' % (lstm_layer_size, window_size)
model = Sequential(name=model_name)
model.add(LSTM(lstm_layer_size, input_shape=(window_size, character_set_length)))
model.add(Dropout(0.5))
model.add(Dense(character_set_length))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

train_episodes, val_episodes = train_test_split(episode_texts, train_size=0.75)

generator = EpisodesSequence(
    episode_texts=train_episodes,
    window_size=window_size,
    step_size=step_size,
    unique_chars=unique_chars,
    char_indices=char_indices,
    batch_size=batch_size
)

validation_generator = EpisodesSequence(
    episode_texts=val_episodes,
    window_size=window_size,
    step_size=step_size,
    unique_chars=unique_chars,
    char_indices=char_indices,
    batch_size=batch_size
)

def on_epoch_end(epoch, logs):
    print('\nWill generate text samples using model %s, after epoch %d' %(model.name, epoch))
    generate_sample_text(
        model,
        episode_texts,
        window_size,
        unique_chars,
        char_indices,
        indices_char
    )

train_model(model, generator, validation_generator, epochs=5,
            on_epoch_end=on_epoch_end, continue_from_checkpoint=True, lr_decay=True)

/home/sumit/Venv/mlpy3venv/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


EpisodesSequence generator initialized. Total windows: 8565022, Total batches: 33458
EpisodesSequence generator initialized. Total windows: 2849562, Total batches: 11132
Checkpoint folder: lstm_256_1layer_wsize_200_2018-03-19_19-29-30
Epoch 1/5
33457/33458 [============================>.] - ETA: 0s - loss: 2.2669
Will generate text samples using model lstm_256_1layer_wsize_200, after epoch 0
--------------------------------------------------
-- diversity: 0.2
-- Generating with seed:
ove past using an md5 hash.  ssl v3 sort of fixed it a little bit by coming up with a clever solution.  they hashed both - they produced both an md5 hash and an sha-1 hash, and they xored the result. 
--
ove past using an md5 hash.  ssl v3 sort of fixed it a little bit by coming up with a clever solution.  they hashed both - they produced both an md5 hash and an sha-1 hash, and they xored the result.  ind the be so the the prome the sere the prome the dere and the prome the prome the sere the pere the the 

with the prescott and all of that.  i can't remember what they called the new stuff.  but they were going to get rid of it.  and then they realized that was a terrible idea.

steve:  was there itanium a long to the ersers and they do it was the problems a like the pass in the security some the securing the because they have to the erach a securing the persesting the security to the compare a like the problems the problems in the security now the compresting the persert the persest in the securing the destated the securing the persersting the security to be the server a long to the emartin on t
--------------------------------------------------
-- diversity: 0.5
-- Generating with seed:
with the prescott and all of that.  i can't remember what they called the new stuff.  but they were going to get rid of it.  and then they realized that was a terrible idea.

steve:  was there itanium
--
with the prescott and all of that.  i can't remember what they called the new stuff.  but they were g

ings.  i rebooted my server.  leo, you and i were talking a week ago about how, you know, i - now if you go to ssl labs and check out grc.com, i got a better grade than i predicted i would a week ago.  ho really got in the securing the pissware is interne to renex.  i went to the pertast the persally be the read the privers of the craching the problem a security, they have to the server because they did the example possing on the problem in the ever because it's like, it's because there is the because it's the end the server so the pight.  i think it's he s art want to read like any our the br
--------------------------------------------------
-- diversity: 1.0
-- Generating with seed:
ings.  i rebooted my server.  leo, you and i were talking a week ago about how, you know, i - now if you go to ssl labs and check out grc.com, i got a better grade than i predicted i would a week ago.
--
ings.  i rebooted my server.  leo, you and i were talking a week ago about how, you know, i - now if 

### Bidirectional LSTM model
This is similar to the previous model, but uses a Bidirectional LSTM. A Bidirectional LSTM is just 2 passes of LSTM executed in forward and backward direction over the input.

1. It has one Bidirectional LSTM layer with 256 hidden units.  
2. We add a dropout of 0.5 to avoid over-fitting.  
3. We add a densely connected layer with output dimension equal to number of characters in our corpus.  
4. We finally end with a softmax layer.

In [13]:
window_size=200
step_size=4
batch_size=256
lstm_layer_size=256

character_set_length = len(unique_chars)

model_name = 'Bilstm_%d_1layer_wsize_%d' % (lstm_layer_size, window_size)
model = Sequential(name=model_name)
model.add(Bidirectional(LSTM(lstm_layer_size), input_shape=(window_size, character_set_length)))
model.add(Dropout(0.5))
model.add(Dense(character_set_length))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

train_episodes, val_episodes = train_test_split(episode_texts, train_size=0.75)

generator = EpisodesSequence(
    episode_texts=train_episodes,
    window_size=window_size,
    step_size=step_size,
    unique_chars=unique_chars,
    char_indices=char_indices,
    batch_size=batch_size
)

validation_generator = EpisodesSequence(
    episode_texts=val_episodes,
    window_size=window_size,
    step_size=step_size,
    unique_chars=unique_chars,
    char_indices=char_indices,
    batch_size=batch_size
)

def on_epoch_end(epoch, logs):
    print('\nWill generate text samples using model %s, after epoch %d' %(model.name, epoch))
    generate_sample_text(
        model,
        episode_texts,
        window_size,
        unique_chars,
        char_indices,
        indices_char
    )

train_model(model, generator, validation_generator, epochs=2,
            on_epoch_end=on_epoch_end, continue_from_checkpoint=True, lr_decay=True)

/home/sumit/Venv/mlpy3venv/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


EpisodesSequence generator initialized. Total windows: 8486766, Total batches: 33152
EpisodesSequence generator initialized. Total windows: 2927817, Total batches: 11437
Checkpoint folder: Bilstm_256_1layer_wsize_200_2018-03-20_02-11-51
Epoch 1/2
33151/33152 [============================>.] - ETA: 0s - loss: 2.2734
Will generate text samples using model Bilstm_256_1layer_wsize_200, after epoch 0
--------------------------------------------------
-- diversity: 0.2
-- Generating with seed:
ere trying as hard as they could to attack another, well, their maximum upstream or outgoing bandwidth would be 256k, or 250k, for example, a quarter of a megabit, which is one quarter of what youre a
--
ere trying as hard as they could to attack another, well, their maximum upstream or outgoing bandwidth would be 256k, or 250k, for example, a quarter of a megabit, which is one quarter of what youre a that the pastine to the ore this is the protered that the to that the partered that the pastions the p

## Observations
1. The generated text starts to feel more coherent as the number of epochs increases.  
1. The temperature parameter used during the text generation phase makes a huge difference.  
    1. When temperature is low (Eg. 0.2), the generated text has less spelling mistakes, but it gets into repetitive loops. Eg.:
    
    ```
    leo:  eric ster, and the saye that the same this is the sone that this a thing to the saye the sant to the parters that the partore of the out that the saye there that the sand to the one the partert that it the pasting to the sare this is the sone that it a a so that's a there that's a come the parters the saye this is a little the sayer of the saying, there's the saye that the saye the out that this is the s
    ```
    
    2. When temperature it set high, the text has more spelling mistakes, but it seems to be less repititive. Eg.:
    
    ```
     i wai it's all i have ecuris machide.  thas was "really, weftrase, a h, i becive.  so twhy eanns id his a sfeve' intecnrtat.  because the okey to ;re fabmebs mich robersing the down the posfage, some noc theabpolisty.  chatige it's necent with the most cancatima grasty ibworgy whie virsof.  gen fannat.
    ```

## References
1. The Unreasonable Effectiveness of Recurrent Neural Networks [[Link]](http://karpathy.github.io/2015/05/21/rnn-effectiveness/).  
2. Keras text generation sample [[Link]](https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py).  
3. Stack Overflow discussion about temperature of Softmax output [[Link]](https://stackoverflow.com/questions/37246030/how-to-change-the-temperature-of-a-softmax-output-in-keras).